In [2]:
# IMPORTANTE ATTENZIONE OCCORRE CONVERTIRE IL FILE DIINGRESSO DEI PUNTI
# IN COORDINATE PIANE METRICHE 

import pandas as pd
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import datetime
from sklearn.cluster import DBSCAN
import mplstereonet
from geopandas import GeoDataFrame
from shapely.geometry import Point
import glob
import os

# Import plotly package
import plotly
plotly.tools.set_credentials_file(username='davide.schenone', api_key='WBSgVI0FXgSIRecK2cHU')
import plotly.graph_objs as go

#from mpl_toolkits.basemap import pyproj
#from mpl_toolkits.basemap import Basemap
from pyproj import Proj, transform


ImportError: No module named mplstereonet

In [3]:
filenames = glob.glob("*.txt")
with open('Terremoti.csv', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

NameError: name 'glob' is not defined

In [4]:
# Check ploltly version
plotly.__version__

# To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py
from pandas import ExcelWriter
from pandas import ExcelFile

# Create data with x and y random over [-2, 2], and z a Gaussian function of x and y.
#np.random.seed(12345)
#x = 2 * (np.random.random(500) - 0.5)
#y = 2 * (np.random.random(500) - 0.5)

#def f(x, y):
 #   return np.exp(-(x + y ** 2))

#z = f(x, y)

#imposto i dati
#importa il file originale
#df = pd.read_csv('Saorge_Taggia_erh_erz_prof.csv', sep='\t', names=['y', 'x','z','a','b'])

#importa il file convertito con convergo
#df = pd.read_excel('dati_terremoti/dati_terremoti.xlsx', sheetname='Foglio1')

df = pd.read_excel('dati_terremoti_completi.xlsx', sheet_name='Foglio1')
df['Data'] = df['Data'].astype(str).str.zfill(6)

#converto le coordinate wgs 84 in UTM epsg 3003 Gauss-Boaga 
x1 = np.array(df['Longit.'])
y1 = np.array(df['Latitud.'])


inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3003')
x2,y2 = transform(inProj,outProj, x1, y1)

df['x'] = x2
df['y'] = y2

df['z'] = df['Prof corr']  #creo il campo z profondità con valori negativi


df['Data'] = df['Data'].astype(str).str.zfill(6) #riempie la mancanza dello zero nei primi valori
df['Data'] =  pd.to_datetime(df['Data'],  format='%y%m%d')
#df['Ora'] =  pd.to_time(df['Ora'],  format='%H%M')
#creo una colonna con  la differenza in giorni tra la data e una posta abbastanza indietro nel passato
df['Deltatime'] = df['Data'] - datetime.date(1989,1,1)
df['Deltatime'] = df['Deltatime'].astype(datetime.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
#con questo posso fare dei cluster temporali
#IMPOSTO I FILTRI

#df_filter = df.query('Erh<1 and Erz<4') #imposto i parametri di filtro sugli errori orizzontali e verticali


#df_filter = df.query('<Deltatime<') #imposto il filtro sulla data

#imposto i pesi per il calcolo del piano da utilizzare in caso di regressione lineare pesata

df['weight_calc'] = df['Nf']/24 - 0.25
def weight(df):
    if df['Nf'] <=6 :
        return 0
    
    elif  df['Nf'] > 6 and df['Nf']  < 30   :
        return df['weight_calc']
            
    elif  df['Nf'] >= 30 :
        return 1

df['weights'] =  df.apply(weight, axis=1) 
 

    
    
df_filter = df

NameError: name 'plotly' is not defined

In [5]:
#df

## Procedo con il PRIMO  clustering attraverso DBSCAN (BASE TEMPORALE)

In [6]:

coords = df_filter.as_matrix(columns=['Deltatime'])



db = DBSCAN(eps=70, min_samples=10).fit(coords)
labels = db.labels_
from collections import Counter


df_filter = df_filter.assign(Classe = labels)

Counter(labels)

NameError: name 'df_filter' is not defined

In [7]:
#inserisco il valore K su cui far girare il ciclo FOR
cicli = df_filter['Classe'].value_counts().index.tolist()
cicli = np.array(cicli)
cicli_clean = np.delete(cicli, np.where(cicli == [-1]), axis=0)# cancello il label -1 che rappresente gli outliers
plot_dip_direction = 0
plot_dip = 0

NameError: name 'df_filter' is not defined

# CICLO FOR

In [8]:
for K in cicli_clean:
    
    
    #filtro la label di interesse
    df_cluster = df_filter[df_filter['Classe'] == K]
    df_filter_2 = df_cluster
    coords_2 = df_cluster.as_matrix(columns=['x','y','z'])

    
    
    #secondo filtraggio con DBSCAN
    db = DBSCAN(eps=3000, min_samples=5).fit(coords_2)
    labels_2 = db.labels_
    from collections import Counter


    df_filter_2 = df_filter_2.assign(Classe_spaziale = labels_2)
    
    #inserisco il valore J su cui far girare il SECONDO ciclo FOR
    cicli_2 = df_filter_2['Classe_spaziale'].value_counts().index.tolist()
    cicli_2 = np.array(cicli_2)
    cicli_2_clean = np.delete(cicli_2, np.where(cicli_2 == [-1]), axis=0)# cancello il label -1 che rappresente gli outliers


 ##################### SECONDO CICLO ######################################   
    
    
    for J in cicli_2_clean:
    
    
        #filtro la label di interesse
        df_cluster_2 = df_filter_2[df_filter_2['Classe_spaziale'] == J]
        
        
        inizio = df_cluster_2.iloc[0,0] # setto la data inizio e fine del cluster
        fine = df_cluster_2.iloc[-1,0]
        
        
        #Creo una mappa dei punti su google maps

        from gmplot import gmplot
        gmap = gmplot.GoogleMapPlotter( 44.5514, 9.4335, 13)

        # Scatter points
        top_attraction_lats = df['Latitud.']
        top_attraction_lons = df['Longit.']


        filter_attraction_lats = df_filter['Latitud.']
        filter_attraction_lons = df_filter['Longit.']

        cluster_attraction_lats = df_cluster_2['Latitud.']# *******ATTENZIONE SONO DATI FILTRATI E CLUSTERIZZATI
        cluster_attraction_lons = df_cluster_2['Longit.']



        gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=100, marker=False)

        #gmap.scatter(filter_attraction_lats, filter_attraction_lons, '#f90707', size=40, marker=False)

        gmap.scatter(cluster_attraction_lats, cluster_attraction_lons, '#1307f9', size=50, marker=False)

        gmap.heatmap(cluster_attraction_lats, cluster_attraction_lons ) # crea una heatmap
        # Draw
        gmap.draw("my_map_time%f%f.html"% (K,J))
    
    
    
    
        #importo il file depurato degli outlayer:
        #df = pd.read_csv('taggia_saorge_ripulita_outlier.csv', sep=',', names=['x', 'y','z'])
        
        x = df_cluster_2['x']  
    
        y = df_cluster_2['y']   
        
        z = df_cluster_2['z']

        W = df_cluster_2['weights']


        
        

        data = np.c_[x,y,z]

        # regular grid covering the domain of the data
        mn = np.min(data, axis=0)
        mx = np.max(data, axis=0)
        X,Y = np.meshgrid(np.linspace(mn[0], mx[0], 20), np.linspace(mn[1], mx[1], 20))
        XX = X.flatten()
        YY = Y.flatten()
    
        # best-fit linear plane (1st-order)
        # applico i pesi

        A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
        L = data[:,2]
        AW = A * np.sqrt(W[:,np.newaxis]) # intermedi con pesi per  calcolo dei coefficienti
        LW = L * np.sqrt(W)               # intermedi con pesi per  calcolo dei coefficienti

        C,_,_,_ = scipy.linalg.lstsq(AW, LW)    # coefficients

        # evaluate it on grid
        Z = C[0]*X + C[1]*Y + C[2]
    
        ####################################################################################################
        #ricavo la direzione e l'immersione del piano attraverso l'equazione del piano e i coseni direttori

        A=C[0]
        B=C[1]
        C_=-1
        D=C[2]

        #calcolo E
        E=np.sqrt(A**2 + B**2 + 1)

        #calcolo i coseni
        cos_a = A/E

        cos_b = B/E

        cos_cp = C_/E


        cos_c = np.cos((np.pi/2) + np.arccos(C_/E))   
        
    
        #print(cos_a, cos_b, cos_c, cos_cp, cos_a_deg, cos_b_deg)


        #calcolo la direzione teta del vettore, il coseni direttori danno la direzione del vettore occorrre aggiungere 90°
        teta_primo = np.arctan(cos_a/cos_b)   #espresso in radianti
        teta_deg = (teta_primo*180)/np.pi 

        #calcolo della direzione della massima pendenza sul piano
        #dip_direction = teta_deg 

        #calcolo l'immersione dip angolo tra la verticale e il piano
        dip_vert = np.arcsin(-1*cos_c)
        dip_deg = (dip_vert*180)/np.pi

        #calcolo l'immersione dip
        dip = dip_deg

        #occorre orientare correttamente il dip_direction
        if cos_a > 0 and cos_b > 0:                
            dip_direction = teta_deg
            if cos_cp <0:
                dip_direction =  dip_direction + 180
        
        if  cos_a > 0 and cos_b < 0:
            dip_direction = teta_deg + 180
            if cos_cp <0:
                dip_direction =  dip_direction + 180
        
            
        if  cos_a < 0 and cos_b < 0:
            dip_direction = teta_deg + 180
            if cos_cp <0:
                dip_direction =  dip_direction - 180
       
    
        if  cos_a < 0 and cos_b > 0:
            dip_direction =  teta_deg + 360
            if cos_cp <0:
                dip_direction =  dip_direction - 180
                 
        
         #creo l'arrai per il plotaggio su stereogramma
        plot_dip_direction = np.append(plot_dip_direction, dip_direction)
        plot_dip = np.append(plot_dip, dip)
        print('##################################################')
        print(K,J)
        print("dip_direction= %f ; dip= %f" % (dip_direction, dip))
        #print(cos_a, cos_b)
     
        output = "teta= %f ; dip= %f" % (dip_direction, dip)
        file = open("giacitura_cluster_%f%f.txt" %(K,J),"w")
        file.write(output)
        file.close()

        daframe_cluster_xx = pd.DataFrame.to_csv(df_cluster_2)
        file = open("daframe_cluster_%f%f.csv" %(K,J),"w")
        file.write(daframe_cluster_xx)
        file.close()
        print("Controllo di aver eliminato scatter outlayer [-1]" , cicli_clean , cicli_2_clean) 
        
        
        #salva i risultati su di un file shape
        geometry = [Point(xy) for xy in zip(df_cluster_2.x , df_cluster_2.y)]
        df_shape = df_cluster_2.drop(['x', 'y','Ora', 'Data', 'Sec'], axis=1)
        crs = {'init': 'epsg:3003'}
        gdf = GeoDataFrame( df_shape, crs=crs, geometry=geometry)   #df_shape,
        gdf.to_file(driver = 'ESRI Shapefile', filename= "result%f%f.shp" %(K,J))

        
        
        #crea gli stereogrammi
strike = plot_dip_direction - 90 
dip =  plot_dip
fig = plt.figure()
ax = fig.add_subplot(111, projection='stereonet')
ax.plane(strike, dip)
ax.pole(strike, dip ,markersize=5)
ax.grid()
plt.show()

#density plot
fig, ax = mplstereonet.subplots()
cax = ax.density_contourf(strike, dip, measurement='poles')
ax.pole(strike, dip)
ax.grid(True)
fig.colorbar(cax)

plt.show()

NameError: name 'cicli_clean' is not defined